In [1]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# flow from directory to numpy array
gen = image.ImageDataGenerator()
img_data = gen.flow_from_directory('../ISIC/ISIC_2018/data_raw/train',
                                  target_size = (224,224),
                                  class_mode = 'categorical',
                                  shuffle = True,
                                  batch_size = 8314)
x, y = img_data.next()

Found 8314 images belonging to 7 classes.


In [3]:
def initialize_model():

    resnet = ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)

    output = resnet.layers[-2].output
    output_layer = Dense(7, activation='softmax', name='predictions')(output)

    custom_resnet = Model(inputs=resnet.input, outputs=output_layer)

    custom_resnet.compile(optimizer=Adam(lr=0.00015),
                loss='categorical_crossentropy', metrics=['accuracy'])
    
    return custom_resnet

In [4]:
# KFold
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(x):

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    custom_resnet = initialize_model()
    
    custom_resnet.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 1, batch_size = 64)

Train on 6651 samples, validate on 1663 samples
Epoch 1/1
6651/6651 [==============================] - 62s 9ms/step - loss: 0.6907 - acc: 0.7516 - val_loss: 0.6767 - val_acc: 0.7583
Train on 6651 samples, validate on 1663 samples
Epoch 1/1
6651/6651 [==============================] - 61s 9ms/step - loss: 0.6765 - acc: 0.7572 - val_loss: 0.6846 - val_acc: 0.7456
Train on 6651 samples, validate on 1663 samples
Epoch 1/1
6651/6651 [==============================] - 61s 9ms/step - loss: 0.6950 - acc: 0.7497 - val_loss: 1.0345 - val_acc: 0.6711
Train on 6651 samples, validate on 1663 samples
Epoch 1/1
6651/6651 [==============================] - 62s 9ms/step - loss: 0.6979 - acc: 0.7510 - val_loss: 0.6090 - val_acc: 0.7643
Train on 6652 samples, validate on 1662 samples
Epoch 1/1
6652/6652 [==============================] - 64s 10ms/step - loss: 0.6917 - acc: 0.7527 - val_loss: 0.7379 - val_acc: 0.7581


In [25]:
# stratified KFold - overall category distribution is maintained across folds
y_cat = np.argmax(y, axis = 1) # convert one hot array to integers
kf = StratifiedKFold(n_splits = 5)

for train_index, test_index in kf.split(x, y_cat):

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    custom_resnet = initialize_model()
    
    custom_resnet.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 1, batch_size = 64)

Train on 6649 samples, validate on 1665 samples
Epoch 1/1
6649/6649 [==============================] - 63s 9ms/step - loss: 0.6948 - acc: 0.7533 - val_loss: 0.6631 - val_acc: 0.7574
Train on 6651 samples, validate on 1663 samples
Epoch 1/1
6651/6651 [==============================] - 61s 9ms/step - loss: 0.6886 - acc: 0.7527 - val_loss: 0.7329 - val_acc: 0.7631
Train on 6651 samples, validate on 1663 samples
Epoch 1/1
6651/6651 [==============================] - 62s 9ms/step - loss: 0.6868 - acc: 0.7537 - val_loss: 0.7435 - val_acc: 0.7619
Train on 6651 samples, validate on 1663 samples
Epoch 1/1
6651/6651 [==============================] - 63s 9ms/step - loss: 0.6577 - acc: 0.7667 - val_loss: 0.7230 - val_acc: 0.7396
Train on 6654 samples, validate on 1660 samples
Epoch 1/1
6654/6654 [==============================] - 65s 10ms/step - loss: 0.6799 - acc: 0.7565 - val_loss: 0.9913 - val_acc: 0.6771
